In [ ]:
from plots import plot_policy_convergence, plot_final_policies, plot_final_policies_linear, plot_specific_state_policy, plot_specific_state_policy_linear
import pickle

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import copy

# Output v0

    U = 2                 
    T = 11            
    delta_t = 1          
    n_travelers = 600
    # KZ: preset initial karma balance
    K = 100 # if this is not enough, the code will crash because out of bound index in the policy state
    k_init = 10
    # Group preference = t_star ∈ {0,1,...,T-1}
    n_groups = 1
    t_star = 8
    phi = np.array([[0.8, 0.2],
                    [0.8, 0.2]])
    u_value = np.array([1.0, 6.0]) # from the paper
    delta = 0.9 
    eta = 0.1 
    alpha = 1.6 # queueing weight
    beta = 1 # early arrival weight
    gamma = 4 # late arrival weight
    ...
    fast_lane_capacity = 12 # MM from the paper
    slow_lane_capacity = 48

In [ ]:
# with open('../groups.pkl', 'rb') as f:
#     groups = pickle.load(f)

# with open('../error_vec.pkl', 'rb') as f:
#     error_vec = pickle.load(f)

# with open('../simulation_params.pkl', 'rb') as f:
#     n_day, n_groups, K, n_travelers = pickle.load(f)

with open('output_v0/groups.pkl', 'rb') as f:
    groups = pickle.load(f)

with open('output_v0/error_vec.pkl', 'rb') as f:
    error_vec = pickle.load(f)

with open('output_v0/simulation_params.pkl', 'rb') as f:
    n_day, n_groups, K, n_travelers = pickle.load(f)

with open('output_v0/system.pkl', 'rb') as f:
    system = pickle.load(f)

In [ ]:
plot_policy_convergence(error_vec, n_day, n_groups)

In [ ]:
old_pi = groups[0].pi 
old_Q = groups[0].Q

In [ ]:
old_b_star = system.b_star
old_slow_lane_queue = system.slow_lane_queue
old_psi = system.psi

In [ ]:
U = 2                 
K = 100    
T = 11

In [ ]:
travelers = groups[0].travelers

In [ ]:
# one day simulation 
# 1. Travelers act
for tr in travelers:
    tr.store_start_state()
    tr.action()

# 2. System queues
system.simulate_lane_queue()

# 3. Payment
for tr in travelers:
    tr.paid_karma_bid()

# 4. Redistribution
system.karma_redistribution()

# 5. Update urgency
for tr in travelers:
    tr.update_urgency()

# Update each group (independent policies)
g = groups[0]
g.update_transition_matrix()
g.update_policy(system)

In [ ]:
# difference in policy by state
np.round(np.linalg.norm(g.pi - old_pi, axis=1), 2)

In [ ]:
np.round(np.linalg.norm(g.pi, axis=1), 2)

In [ ]:
# plot policy by state 
u = 0
k = 0

idx = u * (K+1) + k
old_pi_i = old_pi[idx].reshape(T, K+1).T
pi_i = g.pi[idx].reshape(T, K+1).T

# pi_i.shape
plt.subplot(1,2,1)
plt.imshow(old_pi_i[:12])
plt.subplot(1,2,2)
plt.imshow(pi_i[:12])

In [ ]:
# plot policy by state 
u = 1
k = 0

idx = u * (K+1) + k
old_pi_i = old_pi[idx].reshape(T, K+1).T
pi_i = g.pi[idx].reshape(T, K+1).T

# pi_i.shape
plt.subplot(1,2,1)
plt.imshow(old_pi_i[:12])
plt.subplot(1,2,2)
plt.imshow(pi_i[:12])

In [ ]:
# plot policy by state 
u = 0
k = 5

idx = u * (K+1) + k
old_pi_i = old_pi[idx].reshape(T, K+1).T
pi_i = g.pi[idx].reshape(T, K+1).T

# pi_i.shape
plt.subplot(1,2,1)
plt.imshow(old_pi_i[:12])
plt.subplot(1,2,2)
plt.imshow(pi_i[:12])

In [ ]:
# plot policy by state 
u = 1
k = 5

idx = u * (K+1) + k
old_pi_i = old_pi[idx].reshape(T, K+1).T
pi_i = g.pi[idx].reshape(T, K+1).T

# pi_i.shape
plt.subplot(1,2,1)
plt.imshow(old_pi_i[:12])
plt.subplot(1,2,2)
plt.imshow(pi_i[:12])

In [ ]:
# system dynamics
old_b_star, system.b_star

# Output v1

    U = 2                 
    T = 11            
    delta_t = 1          
    n_travelers = 600
    # KZ: preset initial karma balance
    K = 100 # if this is not enough, the code will crash because out of bound index in the policy state
    k_init = 10
    # Group preference = t_star ∈ {0,1,...,T-1}
    n_groups = 1
    t_star = 8
    phi = np.array([[0.8, 0.2],
                    [0.8, 0.2]])
    u_value = np.array([1.0, 6.0]) # from the paper
    delta = 0.9 
    eta = 0.01 
    alpha = 1.6 # queueing weight
    beta = 1 # early arrival weight
    gamma = 4 # late arrival weight
    ...
    fast_lane_capacity = 12 # MM from the paper
    slow_lane_capacity = 48

eta = 0.01

In [ ]:
# with open('../groups.pkl', 'rb') as f:
#     groups = pickle.load(f)

# with open('../error_vec.pkl', 'rb') as f:
#     error_vec = pickle.load(f)

# with open('../simulation_params.pkl', 'rb') as f:
#     n_day, n_groups, K, n_travelers = pickle.load(f)

with open('output_v1/groups.pkl', 'rb') as f:
    groups = pickle.load(f)

with open('output_v1/error_vec.pkl', 'rb') as f:
    error_vec = pickle.load(f)

with open('output_v1/simulation_params.pkl', 'rb') as f:
    n_day, n_groups, K, n_travelers = pickle.load(f)

with open('output_v1/system.pkl', 'rb') as f:
    system = pickle.load(f)

In [ ]:
plot_policy_convergence(error_vec, n_day, n_groups)

In [ ]:
old_pi = groups[0].pi 
old_Q = groups[0].Q

In [ ]:
old_b_star = system.b_star
old_slow_lane_queue = system.slow_lane_queue
old_psi = system.psi

In [ ]:
U = 2                 
K = 100    
T = 11

In [ ]:
travelers = groups[0].travelers

In [ ]:
# one day simulation 
# 1. Travelers act
for tr in travelers:
    tr.store_start_state()
    tr.action()

# 2. System queues
system.simulate_lane_queue()

# 3. Payment
for tr in travelers:
    tr.paid_karma_bid()

# 4. Redistribution
system.karma_redistribution()

# 5. Update urgency
for tr in travelers:
    tr.update_urgency()

# Update each group (independent policies)
g = groups[0]
g.update_transition_matrix()
g.update_policy(system)

In [ ]:
# difference in policy by state
np.round(np.linalg.norm(g.pi - old_pi, axis=1), 2)

In [ ]:
# plot policy by state 
u = 0
k = 0

idx = u * (K+1) + k
old_pi_i = old_pi[idx].reshape(T, K+1).T
pi_i = g.pi[idx].reshape(T, K+1).T

# pi_i.shape
plt.subplot(1,2,1)
plt.imshow(old_pi_i[:12])
plt.subplot(1,2,2)
plt.imshow(pi_i[:12])

In [ ]:
# plot policy by state 
u = 1
k = 0

idx = u * (K+1) + k
old_pi_i = old_pi[idx].reshape(T, K+1).T
pi_i = g.pi[idx].reshape(T, K+1).T

# pi_i.shape
plt.subplot(1,2,1)
plt.imshow(old_pi_i[:12])
plt.subplot(1,2,2)
plt.imshow(pi_i[:12])

In [ ]:
# plot policy by state 
u = 0
k = 5

idx = u * (K+1) + k
old_pi_i = old_pi[idx].reshape(T, K+1).T
pi_i = g.pi[idx].reshape(T, K+1).T

# pi_i.shape
plt.subplot(1,2,1)
plt.imshow(old_pi_i[:12])
plt.subplot(1,2,2)
plt.imshow(pi_i[:12])

In [ ]:
# plot policy by state 
u = 1
k = 5

idx = u * (K+1) + k
old_pi_i = old_pi[idx].reshape(T, K+1).T
pi_i = g.pi[idx].reshape(T, K+1).T

# pi_i.shape
plt.subplot(1,2,1)
plt.imshow(old_pi_i[:12])
plt.subplot(1,2,2)
plt.imshow(pi_i[:12])

In [ ]:
# system dynamics
old_b_star, system.b_star

# Output v2

    U = 2                 
    T = 11            
    delta_t = 1          
    n_travelers = 600
    # KZ: preset initial karma balance
    K = 100 # if this is not enough, the code will crash because out of bound index in the policy state
    k_init = 10
    # Group preference = t_star ∈ {0,1,...,T-1}
    n_groups = 1
    t_star = 8
    phi = np.array([[0.8, 0.2],
                    [0.8, 0.2]])
    u_value = np.array([1.0, 6.0]) # from the paper
    delta = 0.9 
    eta = 0.1 
    alpha = 1.6 # queueing weight
    beta = 1 # early arrival weight
    gamma = 4 # late arrival weight
    ...
    fast_lane_capacity = 12 # MM from the paper
    slow_lane_capacity = 48

increase the number of days to 500

In [ ]:
# with open('../groups.pkl', 'rb') as f:
#     groups = pickle.load(f)

# with open('../error_vec.pkl', 'rb') as f:
#     error_vec = pickle.load(f)

# with open('../simulation_params.pkl', 'rb') as f:
#     n_day, n_groups, K, n_travelers = pickle.load(f)

with open('output_v2/groups.pkl', 'rb') as f:
    groups = pickle.load(f)

with open('output_v2/error_vec.pkl', 'rb') as f:
    error_vec = pickle.load(f)

with open('output_v2/simulation_params.pkl', 'rb') as f:
    n_day, n_groups, K, n_travelers = pickle.load(f)

with open('output_v2/system.pkl', 'rb') as f:
    system = pickle.load(f)

In [ ]:
plot_policy_convergence(error_vec, n_day, n_groups)

In [ ]:
old_pi = groups[0].pi 
old_Q = groups[0].Q

In [ ]:
old_b_star = system.b_star
old_slow_lane_queue = system.slow_lane_queue
old_psi = system.psi

In [ ]:
U = 2                 
K = 100    
T = 11

In [ ]:
travelers = groups[0].travelers

In [ ]:
# one day simulation 
# 1. Travelers act
for tr in travelers:
    tr.store_start_state()
    tr.action()

# 2. System queues
system.simulate_lane_queue()

# 3. Payment
for tr in travelers:
    tr.paid_karma_bid()

# 4. Redistribution
system.karma_redistribution()

# 5. Update urgency
for tr in travelers:
    tr.update_urgency()

# Update each group (independent policies)
g = groups[0]
g.update_transition_matrix()
g.update_policy(system)

In [ ]:
# difference in policy by state
np.round(np.linalg.norm(g.pi - old_pi, axis=1), 2)

In [ ]:
# plot policy by state 
u = 0
k = 0

idx = u * (K+1) + k
old_pi_i = old_pi[idx].reshape(T, K+1).T
pi_i = g.pi[idx].reshape(T, K+1).T

# pi_i.shape
plt.subplot(1,2,1)
plt.imshow(old_pi_i[:12])
plt.subplot(1,2,2)
plt.imshow(pi_i[:12])

In [ ]:
old_Q = old_Q.reshape(U * (K+1), T * (K+1))
Q = g.Q.reshape(U * (K+1), T * (K+1))

In [ ]:
old_Q[idx].reshape(T, K+1)[:,0], Q[idx].reshape(T, K+1)[:,0]

In [ ]:
# plot policy by state 
u = 1
k = 0

idx = u * (K+1) + k
old_pi_i = old_pi[idx].reshape(T, K+1).T
pi_i = g.pi[idx].reshape(T, K+1).T

# pi_i.shape
plt.subplot(1,2,1)
plt.imshow(old_pi_i[:12])
plt.subplot(1,2,2)
plt.imshow(pi_i[:12])

In [ ]:
old_Q = old_Q.reshape(U * (K+1), T * (K+1))
Q = g.Q.reshape(U * (K+1), T * (K+1))

In [ ]:
old_Q[idx].reshape(T, K+1)[:,0], Q[idx].reshape(T, K+1)[:,0]

In [ ]:
# plot policy by state 
u = 0
k = 5

idx = u * (K+1) + k
old_pi_i = old_pi[idx].reshape(T, K+1).T
pi_i = g.pi[idx].reshape(T, K+1).T

# pi_i.shape
plt.subplot(1,2,1)
plt.imshow(old_pi_i[:12])
plt.subplot(1,2,2)
plt.imshow(pi_i[:12])

In [ ]:
# plot policy by state 
u = 1
k = 5

idx = u * (K+1) + k
old_pi_i = old_pi[idx].reshape(T, K+1).T
pi_i = g.pi[idx].reshape(T, K+1).T

# pi_i.shape
plt.subplot(1,2,1)
plt.imshow(old_pi_i[:12])
plt.subplot(1,2,2)
plt.imshow(pi_i[:12])

In [ ]:
# system dynamics
old_b_star, system.b_star

In [ ]:
old_psi, system.psi

In [ ]:
old_slow_lane_queue, system.slow_lane_queue

# Output V3 (100 days)

In [ ]:
with open('output_V3/groups.pkl', 'rb') as f:
    groups = pickle.load(f)

with open('output_V3/error_vec.pkl', 'rb') as f:
    error_vec = pickle.load(f)

with open('output_V3/simulation_params.pkl', 'rb') as f:
    n_day, n_groups, K, n_travelers = pickle.load(f)

with open('output_V3/system.pkl', 'rb') as f:
    system = pickle.load(f)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

n_iter = 4
u_values = [0, 1]
k = 10

fig, axes = plt.subplots(
    n_iter, 2,
    figsize=(9, 2.3 * n_iter),
    sharex=True, sharey=True
)

for it in range(n_iter):

    # ---- Store policy BEFORE update ----
    pi_before = {}
    for u in u_values:
        idx = u * (K + 1) + k
        pi_before[u] = groups[0].pi[idx].reshape(T, K+1).T

    # ---- One-day simulation ----
    travelers = groups[0].travelers

    for tr in travelers:
        tr.store_start_state()
        tr.action()

    system.simulate_lane_queue()

    for tr in travelers:
        tr.paid_karma_bid()

    system.karma_redistribution()

    for tr in travelers:
        tr.update_urgency()

    for g in groups:
        g.update_policy(system)
        g.update_transition_matrix()

    # ---- Plot ----
    for col, u in enumerate(u_values):
        ax = axes[it, col]

        im = ax.imshow(
            pi_before[u][:11],
            origin="lower",
            aspect="auto",
            vmin=0,
            vmax=0.1
        )

        if it == 0:
            ax.set_title(f"Urgency $u={u}$")

        if col == 0:
            ax.set_ylabel(f"Iter {it}\nBid $b$")

        if it == n_iter - 1:
            ax.set_xlabel("Departure time $t$")

# ---- Manual layout control ----
plt.subplots_adjust(
    left=0.08,
    right=0.88,
    top=0.92,
    bottom=0.08,
    hspace=0.35,
    wspace=0.15
)

# ---- Dedicated colorbar axis ----
cax = fig.add_axes([0.90, 0.15, 0.02, 0.7])
cbar = fig.colorbar(im, cax=cax)
cbar.set_label("Policy probability")

plt.show()


In [ ]:
plot_policy_convergence(error_vec, n_day, n_groups)

In [ ]:
plot_final_policies(groups, n_groups)

In [ ]:
plot_final_policies_linear(groups, n_groups)

In [ ]:
specific_u = 0
specific_k = 3

plot_specific_state_policy(groups, n_groups, K, specific_u, specific_k)

In [ ]:
specific_u = 1
plot_specific_state_policy(groups, n_groups, K, specific_u, specific_k)

# Output V4 (more days)

In [ ]:
with open('output_V4/groups.pkl', 'rb') as f:
    groups = pickle.load(f)

with open('output_V4/error_vec.pkl', 'rb') as f:
    error_vec = pickle.load(f)

with open('output_V4/simulation_params.pkl', 'rb') as f:
    n_day, n_groups, K, n_travelers = pickle.load(f)

with open('output_V4/system.pkl', 'rb') as f:
    system = pickle.load(f)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

n_iter = 4
u_values = [0, 1]
k = 10

fig, axes = plt.subplots(
    n_iter, 2,
    figsize=(9, 2.3 * n_iter),
    sharex=True, sharey=True
)

for it in range(n_iter):

    # ---- Store policy BEFORE update ----
    pi_before = {}
    for u in u_values:
        idx = u * (K + 1) + k
        pi_before[u] = groups[0].pi[idx].reshape(T, K+1).T

    # ---- One-day simulation ----
    travelers = groups[0].travelers

    for tr in travelers:
        tr.store_start_state()
        tr.action()

    system.simulate_lane_queue()

    for tr in travelers:
        tr.paid_karma_bid()

    system.karma_redistribution()

    for tr in travelers:
        tr.update_urgency()

    for g in groups:
        g.update_policy(system)
        g.update_transition_matrix()

    # ---- Plot ----
    for col, u in enumerate(u_values):
        ax = axes[it, col]

        im = ax.imshow(
            pi_before[u][:11],
            origin="lower",
            aspect="auto",
            vmin=0,
            vmax=0.1
        )

        if it == 0:
            ax.set_title(f"Urgency $u={u}$")

        if col == 0:
            ax.set_ylabel(f"Iter {it}\nBid $b$")

        if it == n_iter - 1:
            ax.set_xlabel("Departure time $t$")

# ---- Manual layout control ----
plt.subplots_adjust(
    left=0.08,
    right=0.88,
    top=0.92,
    bottom=0.08,
    hspace=0.35,
    wspace=0.15
)

# ---- Dedicated colorbar axis ----
cax = fig.add_axes([0.90, 0.15, 0.02, 0.7])
cbar = fig.colorbar(im, cax=cax)
cbar.set_label("Policy probability")

plt.show()
